# 📓 Text to Text Quickstart

In this quickstart you will create a simple text to text application and learn how to log it and get feedback.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/quickstart/text2text_quickstart.ipynb)

## Setup
### Add API keys
For this quickstart you will need an OpenAI Key.

In [ ]:
# ! pip install trulens_eval openai

In [12]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."

### Import from TruLens

In [13]:
# Create openai client
from openai import OpenAI
client = OpenAI()

# Imports main tools:
from trulens_eval import Feedback, OpenAI as fOpenAI, Tru
tru = Tru()


### Create Simple Text to Text Application

This example uses a bare bones OpenAI LLM, and a non-LLM just for demonstration purposes.

In [26]:
def llm_standalone(prompt):
    return client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": "You are a question and answer bot, and you answer super upbeat."},
            {"role": "user", "content": prompt}
        ]
    ).choices[0].message.content
    
def llm_standalone_2(prompt):
    return "stuff"

### Send your first request

## Initialize Feedback Function(s)

In [27]:
# Initialize OpenAI-based feedback function collection class:
fopenai = fOpenAI()

# Define a relevance function from openai
f_answer_relevance = Feedback(fopenai.relevance).on_input_output()

✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


## Instrument the callable for logging with TruLens

In [33]:
from trulens_eval import TruBasicApp

# Daniel - notice a regression behavior here, where only the recorder defined last (tru_llm_standalone_recorder_2) will be recording, anything above does not perform recording at all.
tru_llm_standalone_recorder = TruBasicApp(llm_standalone, app_id="Happy Bot 1", feedbacks=[f_answer_relevance])

tru_llm_standalone_recorder_2 = TruBasicApp(llm_standalone_2, app_id="Happy Bot 2", feedbacks=[f_answer_relevance])


In [30]:
prompt_input = "what is language models"

In [32]:
tru.reset_database()

In [ ]:
with tru_llm_standalone_recorder as recording:
    tru_llm_standalone_recorder.app(prompt_input)

recording.get() # Daniel: this is not adding records to the DB
with tru_llm_standalone_recorder_2 as recording:
    tru_llm_standalone_recorder_2.app(prompt_input)

## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

## Or view results directly in your notebook

In [11]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,relevance,relevance_calls,latency,total_tokens,total_cost
0,Happy Bot 2,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_e0b15a13df4f519a37f348a905ad3d15,"""How good is language AI?""","""Language AI is absolutely amazing! It has com...",-,"{""record_id"": ""record_hash_e0b15a13df4f519a37f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-07T14:43:28.236744"", ""...",2024-06-07T14:43:29.270988,0.8,[{'args': {'prompt': 'How good is language AI?...,1,68,0.000121
